In [ ]:
!pwd

In [ ]:
# pip install plotly
random_state=42

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import plotly.express as px

np.random.seed(42)

In [ ]:
from tensorflow.keras.layers import Dropout,Conv1D,MaxPooling1D,Flatten, Activation, Dense
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
from datetime import datetime
from tensorflow.keras.layers import Dropout,Conv1D,MaxPooling1D,Flatten, Activation
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from autosklearn.regression import AutoSklearnRegressor
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L2,L1,L1L2
import tensorflow as tf


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import plotly.express as px
import calendar
from datetime import datetime




import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Input, Dense, LSTM, Bidirectional, Dropout
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, auc, classification_report
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt


In [ ]:
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    explained_variance_score,
    accuracy_score,
    r2_score,
)
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    recall_score,
    cohen_kappa_score,
    matthews_corrcoef,
    precision_recall_fscore_support,
)

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold, GroupKFold, LeaveOneGroupOut
import joblib
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay
#     RocCurveDisplay.from_estimator(model, X_vals, y_vals)



from sklearn.metrics import classification_report

from sklearn.metrics import classification_report


from sklearn.model_selection import GroupShuffleSplit

from sklearn.metrics import f1_score,classification_report

In [ ]:

%load_ext tensorboard
%reload_ext tensorboard


def sk_agregated_score(model, X_test, y_test,threshold, ratio_threshold):
    #For testing, input whole test at a time
    # threshold: threshold for individual predictions
    # ratio_threshold: ratio of positive to negative predictions required

    Q_y_test=[]
    Q_y_pred=[]
    Q_stoped_window=[]
    Q_total_windows=[]
    Q_test_name=[]
    for p,x in enumerate(zip(metadata[metadata['Type']==dataframe_type].iloc[:,2],metadata[metadata['Type']==dataframe_type].iloc[:,3],metadata[metadata['Type']==dataframe_type].iloc[:,1])):
        try:
            _X_test=X_test[x[0]:x[1]] #ONLY ONE TEST
            _y_test=y_test[x[0]:x[1]]#ONLY ONE TEST

            test_name=x[2]#metadata[metadata['Type']==dataframe_type].iloc[x,2] #1 String

            # count of positive and negative predictions for each data point
            pos_counts = 0    
            neg_counts = 0
            ratio=1

            total_windows=len(_y_test)

            Q_y_test.append(_y_test[0])
            Q_total_windows.append(total_windows)
            Q_test_name.append(test_name)




            for j in range(0,_y_test.shape[0],1):
#                     print("!!!!!!!!!!!PREVIOUSCHECKPOINT!!!!!!!!!!!!!!!")
                prob = model.predict(_X_test[j].reshape(1,_X_test[j].shape[0] ),verbose=0)

                prob=prob[0][0]

                if prob >= threshold:

                    pos_counts += 1
                else:
                    neg_counts += 1


                if neg_counts == 0:
                    ratio = float('inf') # set the ratio to infinity if there are no negative counts
                else:
                    ratio = pos_counts / neg_counts

                # classify the entire sample as class 1 and move on to the next sample
                if ratio >= ratio_threshold and ratio != float('inf'):

                    y_pred = 1
                    stoped_window=j

                    break     


                else:

                    y_pred = 0
                    stoped_window=j

            Q_y_pred.append(y_pred)
            if y_pred==1:

                print("Stopped at: ",stoped_window)
            else:
                print("No Chatter Detected")
            Q_stoped_window.append(stoped_window)

        except Exception as e:
            pass
    return Q_y_test,Q_y_pred,Q_stoped_window,Q_total_windows,Q_test_name







def sk_agregated_testing_score(model,test_inputs,test_out):
    pass


#SEQUENCE GENERATOR

def gen_seq(data, seq_length, columns):
    L=[]
    n = data.shape[0]
    for i in range(0, n - seq_length,JUMPING_STEP):
        try:
            L.append(data[i:i+seq_length][columns].values)
            if data[i:i+seq_length][columns].values.shape[0] != 100:
                print(data[i:i+seq_length][columns].values.shape)
        except:

            break

    return (L)


#OUTPUTS
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements:JUMPING_STEP]


# evaluate a given model using cross-validation

def cross_validation_function(selected_features,utc_time,random_state):
    
    #THIS DATA JUST RECEIVED HAS TO BECOME THE SEQUENCES
    #WE UNITE THE SEQUENCES AGAIN LET THE K FOLD HANDLE THE SPLIT
    

    
    data_x = selected_features.iloc[:,2:-3]
    data_y = selected_features.iloc[:,-4:-2]
    
    
#     print(training_x.tail())
#     print(training_y.tail())
#     input()


    gkf = GroupKFold(n_splits=FOLDS[0])
    groups= selected_features['Filename']
    lst_accu_stratified = []
    

    for train_index, test_index in gkf.split(data_x,data_y,groups):
        print("IN")
        _X_train, X_test = data_x.iloc[train_index], data_x.iloc[test_index]
        print("SECOND")
        _y_train, y_test = data_y.iloc[train_index], data_y.iloc[test_index]
        
        
        #I GET MY DATASETS SEPARATED THEN I CONSTRUCT THE SEQUENCES:
        ###SECOND PART!1!!
        #TODO: PERFORM THE SPLIT WITH 5 KFOLD! OVER THE TRINING DATA PREVIOSLY SPLITTED
        #NOW WE PERFORM THE SPLITS FROM OUTSIDE
        
        
        #TO CrEAtE thE VALIDATION SPLIT
        splitter = GroupShuffleSplit(
        test_size=.10,
        n_splits=2,
        random_state=random_state)
        split = splitter.split(_X_train,_y_train, groups=_y_train['Filename'])
        train_inds, vals_inds = next(split)

        X_train,X_vals = _X_train.iloc[train_inds], _X_train.iloc[vals_inds]#CHANGED JT
        y_train,y_vals = _y_train.iloc[train_inds], _y_train.iloc[vals_inds]#CHANGED JT
        
#         print(X_train.shape)
#         print(X_train.head())
#         input()
        



        #THIS TRAIN WILL HAVE A SUFFLED SEQUENCE BASED SPLIT FOR THE INNNER VALIDATION IN THE MODEL!
        train_inputs = np.vstack([gen_seq(X_train[X_train['Filename'] == id], seq_length, ["1-1: Channel 1 (Pa)","Filename"])
            for id in X_train['Filename'].unique()])


        test_inputs = np.vstack([gen_seq(X_test[X_test['Filename'] == id], seq_length, ["1-1: Channel 1 (Pa)","Filename"])
                            for id in X_test['Filename'].unique()])


        #THIS ONE GOES FOR SK_AGGREGATION
        vals_inputs = np.vstack([gen_seq(X_vals[X_vals['Filename'] == id], seq_length, ["1-1: Channel 1 (Pa)","Filename"])
                            for id in X_vals['Filename'].unique()])




        train_out = np.vstack([gen_labels(y_train[y_train['Filename'] == id], seq_length, ["IS_UNSTABLE"] )
                            for id in y_train['Filename'].unique()])

        test_out = np.vstack([gen_labels(y_test[y_test['Filename'] == id], seq_length, ["IS_UNSTABLE"])
                            for id in y_test['Filename'].unique()])

        vals_out = np.vstack([gen_labels(y_vals[y_vals['Filename'] == id], seq_length, ["IS_UNSTABLE"])
                            for id in y_vals['Filename'].unique()])




        #NEW SPLITTING!!!   WE KEEP WHOLE SEQUENCES SHUFFLE SEQUENCES

        # set aside 20% of train and test data for evaluation HERE WE SHUFFLE OUR SEQUENCES
#         train_inputs, test_inputs, train_out, test_out = train_test_split(train_inputs, train_out,
#             test_size=0.25, shuffle = True, random_state = random_state)

    #     # Use the same function above for the validation set WE JUST SPLIT IT IN 0.25 and 0.75 OF THE PREVIOUS SPLIT
        train_inputs, vals_inputs_inner, train_out, vals_out_inner = train_test_split(train_inputs, train_out, 
            test_size=0.25,shuffle=True, random_state= random_state) # 0.25 x 0.8 = 0.2






        #DROPPING FILENAME ONLY FOR WHAT I WILL USE NOW

        print(train_inputs.shape)
        train_inputs=train_inputs[:,:,:-1]
        vals_inputs_inner=vals_inputs_inner[:,:,:-1]
        print(train_inputs.shape)



        print("train_out")
        print(train_out.shape)
        scaler=MinMaxScaler(feature_range=(0, 1))
        train_out=scaler.fit_transform(train_out)
        train_out=train_out.reshape(-1,1)



        
        print("vals_out_inner")
        print(vals_out_inner.shape)
        scaler=MinMaxScaler(feature_range=(0, 1))
        vals_out_inner=scaler.fit_transform(vals_out_inner)
        vals_out_inner=vals_out_inner.reshape(-1,1)

        
        
        train_inputs=train_inputs.astype('float32')
        vals_inputs_inner=vals_inputs_inner.astype('float32')
        
        train_out=train_out.astype('float32')


        vals_out_inner=vals_out_inner.astype('float32')
        

        print("train_inputs:",train_inputs.shape)

        print("vals_inputs_inner:",vals_inputs_inner.shape)
        
        print("train_out:",train_out.shape)

        print("vals_out_inner:",vals_out_inner.shape)
        
        

            
        path='/home/jupyter/DataProject'

        log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        print(log_dir)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

        tf.keras.backend.clear_session()
        conf = tf.compat.v1.ConfigProto()
        conf.gpu_options.allow_growth=True
        session = tf.compat.v1.Session(config=conf)


        nb_features = train_inputs.shape[2]
        sequence_length  = train_inputs.shape[1]
        nb_out = train_out.shape[1]

        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv1D(filters=32,
                       kernel_size=8,
                       strides=1,
                       activation='relu',
                       padding='same'),
            tf.keras.layers.LSTM(64, input_shape = (sequence_length, nb_features), return_sequences = True),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.LSTM(32),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(nb_out, activation = 'sigmoid')
        ])

        lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 10**-7 * 10**(epoch/3))

        model.compile(loss=tf.keras.losses.Huber(), optimizer = tf.keras.optimizers.Adam(lr = 10**-7), metrics =['mse','mae'])

        es = EarlyStopping(monitor="val_loss",
                mode="auto",
                verbose=2,
                patience=20,
                min_delta=0.0001,
                restore_best_weights=True)





        history=model.fit(train_inputs,train_out,epochs=100,validation_data= (vals_inputs_inner,vals_out_inner) ,verbose=2,callbacks=[tensorboard_callback,lr,es])

        

        #THIS FUNCTION SAVES THE BEST THRESHOLD RATION POSSIBLE! RETIREVES THE BEST MODEL AS MODEL AND ALSO SAVES IT! AND METRICS AND BEST PARAMETERS
        #THIS IS PER CV IN THE K-FOLD
        


        

        
        _Q_y_test,_Q_y_pred,_Q_stoped_window,_Q_total_windows,_Q_test_name=sk_agregated_score(model, vals_inputs, vals_out, THRESHOLD, RATIO_THRESHOLD)
        
        
        
        
        #NOW TESTING SCORE
        #THIS DELETES THE FILENAME THEN DEACTIVATE
        #vals_inputs=vals_inputs[:,:,:-1]
        
        print("vals_out")
        print(vals_out.shape)
        scaler=MinMaxScaler(feature_range=(0, 1))
        vals_out=scaler.fit_transform(vals_out)
        vals_out=vals_out.reshape(-1,1)
        
        vals_inputs=vals_inputs.astype('float32')
        vals_out=vals_out.astype('float32')
        print("vals_inputs:",vals_inputs.shape)
        print("vals_out:",vals_out.shape)
        
        
        #THIS DELETES THE FILENAME THEN DEACTIVATE
        #test_inputs=test_inputs[:,:,:-1]
        
        print("test_out")
        print(test_out.shape)
        scaler=MinMaxScaler(feature_range=(0, 1))
        test_out=scaler.fit_transform(test_out)
        test_out=test_out.reshape(-1,1)

        test_inputs=test_inputs.astype('float32')
        test_out=test_out.astype('float32')
        print("test_inputs:",test_inputs.shape)
        print("test_out:",test_out.shape)
        
        
        sk_agregated_testing_score(model,test_inputs,test_out) #TO BE IMPLEMENTES IT WILL USE THE TESTING PORTION AND THE MODEL TO PERFORM A TESTING
        
        
        
        
        
        lst_accu_stratified.append("CV ROUND")

    return lst_accu_stratified



In [ ]:
def experiment(FOLDS, THRESHOLD,RATIO_THRESHOLD,PREFIXES,random_state):
    print("###########################NEW_EXPERIMENT_STARTED#############################")
    date = datetime.utcnow()
    utc_time = calendar.timegm(date.utctimetuple())
    print("UTC TIME: ",utc_time)
    

    data = pd.read_csv("transformed_data/transformed_0_final_dataset.csv",index_col=False)
    data=data[data['Filename'].str.contains('data/Demo Tool Makino/') ]


    data['Points'] = range(0,len(data))
    sorted_data = data.sort_values(["Filename",'Points'])
    samples_list = list(sorted_data["Filename"].unique())
    prefixes = PREFIXES
    data=data[data.Filename.str.startswith(tuple(prefixes))]

    ### Preprocessing The Features
    data['Points'] = range(0,len(data))
    sorted_data = data.sort_values(["Filename",'Points'])
    samples_list = list(sorted_data["Filename"].unique())
    selected_features = sorted_data
    
#     stable_subset = selected_features[selected_features['IS_UNSTABLE'] == 0]
#     stable_samples = list((stable_subset['Filename']).unique())


#     unstable_subset = selected_features[selected_features['IS_UNSTABLE'] == 1]
#     unstable_samples = list((unstable_subset['Filename']).unique())

#     splitter = GroupShuffleSplit(
#         test_size=.10,
#         n_splits=2,
#         random_state=random_state,
#     )
#     split = splitter.split(selected_features, groups=selected_features['Filename'])
#     train_inds, test_inds = next(split)

#     _training_dataframe = selected_features.iloc[train_inds]#CHANGED JT
#     testing_dataframe = selected_features.iloc[test_inds]

# #     Training & Validation 

#     splitter = GroupShuffleSplit(
#         test_size=.10,
#         n_splits=2,
#         random_state=random_state,
#     )
#     split = splitter.split(_training_dataframe, groups=_training_dataframe['Filename'])
#     train_inds, val_inds = next(split)

#     training_dataframe = _training_dataframe.iloc[train_inds]
#     validation_dataframe = _training_dataframe.iloc[val_inds]

#     #RESETTING FOR LATER MERGE METADATA WITH THE ACTUAL TESTS
#     training_dataframe=training_dataframe.reset_index()
#     validation_dataframe=validation_dataframe.reset_index()
#     testing_dataframe=testing_dataframe.reset_index()

    
#     ## Indices "Datapoints"

#     _training_dataframe=[]
#     for i in training_dataframe['Filename'].unique():
#         _training_dataframe.append(("training_dataframe",i, training_dataframe[training_dataframe['Filename']==i].index.min(), training_dataframe[training_dataframe['Filename']==i].index.max()))

#     print("#######")
#     _validation_dataframe=[]
#     for i in validation_dataframe['Filename'].unique():
#         _validation_dataframe.append(("validation_dataframe",i, validation_dataframe[validation_dataframe['Filename']==i].index.min(), validation_dataframe[validation_dataframe['Filename']==i].index.max()))

#     print("#######")
#     _testing_dataframe=[]
#     for i in testing_dataframe['Filename'].unique():
#         _testing_dataframe.append(("testing_dataframe",i, testing_dataframe[testing_dataframe['Filename']==i].index.min(), testing_dataframe[testing_dataframe['Filename']==i].index.max()))


#     metadata=pd.concat([pd.DataFrame(_training_dataframe,columns=['Type','Filename','ini_index','end_index']),
#                        pd.DataFrame(_validation_dataframe,columns=['Type','Filename','ini_index','end_index']),
#                        pd.DataFrame(_testing_dataframe,columns=['Type','Filename','ini_index','end_index'])]
#                       )

        
#     #JOIN TRAINING AND VALIDATION!
#     training_dataframe=pd.concat([training_dataframe,validation_dataframe])

#     training_x = training_dataframe.iloc[:,1:-2]
#     training_y = training_dataframe.iloc[:,-4:-2]
    
#     validation_x = validation_dataframe.iloc[:,1:-2]
#     validation_y = validation_dataframe.iloc[:,-4:-2]

#     testing_x = testing_dataframe.iloc[:,1:-2]
#     testing_y = testing_dataframe.iloc[:,-4:-2]
    

#     #to remove the filename
#     X_train, y_train = training_x.iloc[:,0:-1],training_y
#     X_vals, y_vals = validation_x.iloc[:,0:-1],validation_y
#     X_test,y_test = testing_x.iloc[:,0:-1],testing_y
    


    #NEW AGGREGATION TECHNIQUE NOW I PASS ALL SPLITS!!!! TRAIN,VALS,TEST AS DATAFRAMES!!!!
    lst_accu_stratified = cross_validation_function(selected_features,utc_time,random_state)
    print(lst_accu_stratified)


    #HERE!!!!!
    
    
    
    
    
    
    
    
#     Q_y_test=[]
#     Q_y_pred=[]
#     objective=[]
    
#     print("CHECK POINT!!!!!!!!!!!!!!!!!!!")
#     print(lst_accu_stratified) 
#     for i in lst_accu_stratified:

#         objective.append(f1_score(i[0], i[1],average='macro'))
#         print(classification_report(i[0], i[1]))

#     print("Objective",objective)


#     Q_y_test,Q_y_pred,Q_stoped_window,Q_total_windows,Q_test_name=sk_agregated_score(model, X_test, y_test,metadata,"testing_dataframe",  THRESHOLD, RATIO_THRESHOLD)

     
    return "DONE"
# objective,model




In [ ]:
#OPTIMIZATION PARAMETERS #NOW 200 COMBINATIONS!
PREFIXES = [['data/Demo Tool Makino/']]
FOLDS=[3] #KFOLD NUMBER OF FOLDS
THRESHOLD=[i for i in np.arange(0.01,0.85,0.1)] #[0.2,0.3,0.4,0.5,0.6,0.7,0.8] #FROM 0.1111 UP TO 0 0.9999
RATIO_THRESHOLD=[1.5,2,2.5] #HIGHER OR EQUAL THAN 1
seq_length=100
JUMPING_STEP=80
RNDSEED=[39]
np.random.seed(42)

In [ ]:
from typing import Iterable, Any
from __future__ import annotations
from itertools import product

def grid_parameters(parameters: dict[str, Iterable[Any]]) -> Iterable[dict[str, Any]]:
    for params in product(*parameters.values()):
        yield dict(zip(parameters.keys(), params))


parameters ={"FOLDS":FOLDS,"THRESHOLD":THRESHOLD,"RATIO_THRESHOLD":RATIO_THRESHOLD,"PREFIXES":PREFIXES,"random_state":RNDSEED}
rfcs=[]
objectives=[]
for settings in grid_parameters(parameters):
    print(settings)
    objective2,model2 = experiment(**settings)
    print("MODEL APPENDED")
    rfcs.append(model2)
    objectives.append(np.mean(objective2))
    print("NOTE RESUTLTS: ", np.mean(objective2))